In [1]:
import time
import glob as glob
from zipfile import ZipFile
from bs4 import BeautifulSoup
import dateutil.parser as dparser
from datetime import datetime
import pandas as pd
import multiprocessing as mp
import numpy as np

In [2]:
df = pd.read_csv('/home/shane/Documents/thesis/output/final/failures.csv')
df

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,source,service,timestamp_dir,timestamp_site,success_count,success_pct,partial_count,partial_pct,major_count,major_pct,maintenance_count,maintenance_pct
0,slack,slack,2020-03-26 12:00:01,NaN,10,1.0,0,0.0,0,0.0,0,0.0
1,slack,slack,2020-03-26 13:00:01,NaN,10,1.0,0,0.0,0,0.0,0,0.0
2,slack,slack,2020-03-26 14:00:02,NaN,10,1.0,0,0.0,0,0.0,0,0.0
3,slack,slack,2020-03-26 15:00:01,NaN,10,1.0,0,0.0,0,0.0,0,0.0
4,slack,slack,2020-03-26 16:00:01,NaN,10,1.0,0,0.0,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
403998,cloud-google-cloud-platform,google-cloud,2020-06-10 19:00:02,NaN,1,1.0,0,0.0,0,0.0,0,0.0
403999,cloud-google-cloud-platform,google-cloud,2020-06-10 20:00:01,NaN,1,1.0,0,0.0,0,0.0,0,0.0
404000,cloud-google-cloud-platform,google-cloud,2020-06-10 21:00:01,NaN,1,1.0,0,0.0,0,0.0,0,0.0
404001,cloud-google-cloud-platform,google-cloud,2020-06-10 22:00:01,NaN,1,1.0,0,0.0,0,0.0,0,0.0


### Mean of statuses per source/service

In [4]:
df_mean = df.groupby(['source','service']).agg(mean_success=('success_pct',np.mean), mean_partial=('partial_pct',np.mean), mean_major=('major_pct',np.mean), mean_maintenance=('maintenance_pct',np.mean))
df_mean

mean_success  mean_partial  mean_major  \
source        service                                              
atlassian     access          1.000000      0.000000    0.000000   
              bitbucket       0.995298      0.000964    0.000855   
              confluence      0.994699      0.004399    0.000332   
              developers      0.998860      0.000553    0.000449   
              jira-align      1.000000      0.000000    0.000000   
...                                ...           ...         ...   
downrightnow  typepad         1.000000      0.000000    0.000000   
              yahoomail       0.995968      0.004032    0.000000   
              youtube         1.000000      0.000000    0.000000   
github-status github          0.996964      0.002711    0.000325   
slack         slack           0.980163      0.019837    0.000000   

                          mean_maintenance  
source        service                       
atlassian     access              0.000000  
              bitbucket           0.002883  
              confluence          0.000570  
              developers          0.000138  
              jira-align          0.000000  
...                                    ...  
downrightnow  typepad             0.000000  
              yahoomail           0.000000  
              youtube             0.000000  
github-status github              0.000000  
slack         slack               0.000000  

[188 rows x 4 columns]

### Failure MTBF per source/service

In [24]:
for name, group in df.groupby(['source','service']):
    for row in group:
        print(group['timestamp_dir'])

337456    2020-03-26 12:00:01
337460    2020-03-26 13:00:01
337482    2020-03-26 14:00:02
337497    2020-03-26 15:00:01
337507    2020-03-26 16:00:01
                 ...         
361246    2020-06-10 19:00:02
361262    2020-06-10 20:00:01
361272    2020-06-10 21:00:01
361293    2020-06-10 22:00:01
361302    2020-06-10 23:00:01
Name: timestamp_dir, Length: 1835, dtype: object
337456    2020-03-26 12:00:01
337460    2020-03-26 13:00:01
337482    2020-03-26 14:00:02
337497    2020-03-26 15:00:01
337507    2020-03-26 16:00:01
                 ...         
361246    2020-06-10 19:00:02
361262    2020-06-10 20:00:01
361272    2020-06-10 21:00:01
361293    2020-06-10 22:00:01
361302    2020-06-10 23:00:01
Name: timestamp_dir, Length: 1835, dtype: object
337456    2020-03-26 12:00:01
337460    2020-03-26 13:00:01
337482    2020-03-26 14:00:02
337497    2020-03-26 15:00:01
337507    2020-03-26 16:00:01
                 ...         
361246    2020-06-10 19:00:02
361262    2020-06-10 20:00:01
36

6126      2017-10-30 17:00:01
6235      2017-10-30 18:00:01
6431      2017-10-30 19:00:01
6526      2017-10-30 20:00:01
6686      2017-10-30 21:00:01
                 ...         
324857    2018-09-18 03:00:01
324937    2018-09-18 04:00:01
325200    2018-09-18 05:00:01
325298    2018-09-18 06:00:01
325499    2018-09-18 07:00:01
Name: timestamp_dir, Length: 2075, dtype: object
6126      2017-10-30 17:00:01
6235      2017-10-30 18:00:01
6431      2017-10-30 19:00:01
6526      2017-10-30 20:00:01
6686      2017-10-30 21:00:01
                 ...         
324857    2018-09-18 03:00:01
324937    2018-09-18 04:00:01
325200    2018-09-18 05:00:01
325298    2018-09-18 06:00:01
325499    2018-09-18 07:00:01
Name: timestamp_dir, Length: 2075, dtype: object
6126      2017-10-30 17:00:01
6235      2017-10-30 18:00:01
6431      2017-10-30 19:00:01
6526      2017-10-30 20:00:01
6686      2017-10-30 21:00:01
                 ...         
324857    2018-09-18 03:00:01
324937    2018-09-18 04:00:01
32

6140      2017-10-30 17:00:01
6221      2017-10-30 18:00:01
6451      2017-10-30 19:00:01
6544      2017-10-30 20:00:01
6671      2017-10-30 21:00:01
                 ...         
324880    2018-09-18 03:00:01
324997    2018-09-18 04:00:01
325095    2018-09-18 05:00:01
325354    2018-09-18 06:00:01
325479    2018-09-18 07:00:01
Name: timestamp_dir, Length: 2068, dtype: object
6140      2017-10-30 17:00:01
6221      2017-10-30 18:00:01
6451      2017-10-30 19:00:01
6544      2017-10-30 20:00:01
6671      2017-10-30 21:00:01
                 ...         
324880    2018-09-18 03:00:01
324997    2018-09-18 04:00:01
325095    2018-09-18 05:00:01
325354    2018-09-18 06:00:01
325479    2018-09-18 07:00:01
Name: timestamp_dir, Length: 2068, dtype: object
6140      2017-10-30 17:00:01
6221      2017-10-30 18:00:01
6451      2017-10-30 19:00:01
6544      2017-10-30 20:00:01
6671      2017-10-30 21:00:01
                 ...         
324880    2018-09-18 03:00:01
324997    2018-09-18 04:00:01
32

6183      2017-10-30 17:00:01
6273      2017-10-30 18:00:01
6395      2017-10-30 19:00:01
6622      2017-10-30 20:00:01
6704      2017-10-30 21:00:01
                 ...         
324886    2018-09-18 03:00:01
325014    2018-09-18 04:00:01
325223    2018-09-18 05:00:01
325334    2018-09-18 06:00:01
325392    2018-09-18 07:00:01
Name: timestamp_dir, Length: 2078, dtype: object
6183      2017-10-30 17:00:01
6273      2017-10-30 18:00:01
6395      2017-10-30 19:00:01
6622      2017-10-30 20:00:01
6704      2017-10-30 21:00:01
                 ...         
324886    2018-09-18 03:00:01
325014    2018-09-18 04:00:01
325223    2018-09-18 05:00:01
325334    2018-09-18 06:00:01
325392    2018-09-18 07:00:01
Name: timestamp_dir, Length: 2078, dtype: object
6183      2017-10-30 17:00:01
6273      2017-10-30 18:00:01
6395      2017-10-30 19:00:01
6622      2017-10-30 20:00:01
6704      2017-10-30 21:00:01
                 ...         
324886    2018-09-18 03:00:01
325014    2018-09-18 04:00:01
32

6119      2017-10-30 17:00:01
6319      2017-10-30 18:00:01
6373      2017-10-30 19:00:01
6561      2017-10-30 20:00:01
6787      2017-10-30 21:00:01
                 ...         
324836    2018-09-18 03:00:01
324972    2018-09-18 04:00:01
325137    2018-09-18 05:00:01
325373    2018-09-18 06:00:01
325456    2018-09-18 07:00:01
Name: timestamp_dir, Length: 2081, dtype: object
6119      2017-10-30 17:00:01
6319      2017-10-30 18:00:01
6373      2017-10-30 19:00:01
6561      2017-10-30 20:00:01
6787      2017-10-30 21:00:01
                 ...         
324836    2018-09-18 03:00:01
324972    2018-09-18 04:00:01
325137    2018-09-18 05:00:01
325373    2018-09-18 06:00:01
325456    2018-09-18 07:00:01
Name: timestamp_dir, Length: 2081, dtype: object
6119      2017-10-30 17:00:01
6319      2017-10-30 18:00:01
6373      2017-10-30 19:00:01
6561      2017-10-30 20:00:01
6787      2017-10-30 21:00:01
                 ...         
324836    2018-09-18 03:00:01
324972    2018-09-18 04:00:01
32